Psuedocode: 
Create a matrix of length of T/delta T by L/delta X
define a c

we have the initial 


In [31]:
R = 2 #length
T = 1 #Total Time
delta_t = .01 #Timestep
c = 1 #Wavespeed
     
delta_r = .01 #Spatial step
ntsteps = T/delta_t #number of time steps
nrsteps = R/delta_r #number of segments of string
nrsteps = Int(nrsteps)
ntsteps = Int(ntsteps)
using Plots
using FFTW

In [48]:
gamma = 1.2
#Initial Radius
r = zeros(nrsteps+2, 1)
r[1]= -1*delta_r
for i = 1:size(r,1)-1
    r[i+1]=r[i]+delta_r
end

#Time
t=zeros(ntsteps+1,1)
t[1]=0
for i = 1:ntsteps
    t[i+1]=t[i]+delta_t
end

#Equilibrium Quantities

#Equilibrium density
rho_0 = zeros(nrsteps+2,1)
for i = 1:nrsteps+2
    rho_0[i] = 10*sin(pi*r[i]/R)/(r[i]/R)
end

#Equilibrium pressure
p_0 = zeros(nrsteps+2,1)
for i = 1:nrsteps+2
    p_0[i] = rho_0[i]^2
end

#Mass calculation
M = zeros(nrsteps+2,1)
M[1] = 0
for i = 3:nrsteps+1
    M[i+1]=M[i] + (4/3)*pi*(delta_r^3)*rho_0[i]
end

#Equilibrium Gravity
g_0 = zeros(nrsteps+2)
for i=1:nrsteps+2
    g_0[i]=M[i]/(r[i]^2)
end


#Zeta_r for normal intervals
Z_r = zeros(nrsteps+2,ntsteps+2)
for i=1:nrsteps+2
    Z_r[i,1:2].=.04*sin(0.1*pi*r[i])
end

#Initializing the perturbed quantities
rho_prime = zeros(nrsteps+1,ntsteps+2)
p_prime = zeros(nrsteps+1,ntsteps+2)
dp_prime_dr  = zeros(nrsteps+1,ntsteps+2)


#Actual Z calculations
for j = 1:ntsteps

    #rho_prime calculation
    for i = 1:nrsteps+1
        rho_prime[i,j+1] = (-1/((r[i+1]^2)*delta_r))*(((r[i+1]^2)*rho_0[Int(i+1)]*Z_r[i+1,j])-(r[i]^2)*rho_0[i]*Z_r[i,j])
    end
    
   #pprime calculation
    for i = 1:nrsteps+1
        p_prime[i,j] = gamma*p_0[i]/rho_0[i] * rho_prime[i,j]
    end
    
    #dP/dR at normal i
    for i = 1:nrsteps
        dp_prime_dr[i,j] = (p_prime[i+1,j]-p_prime[i,j])/delta_r
    end
    
    #d^2Z/dt^2
    #Boundary condition for the surface
    Z_r[size(Z_r,1),j+2]=Z_r[size(Z_r,1)-1,j+2]-delta_r*(2/r[size(Z_r,1)-1])*Z_r[size(Z_r,1)-1,j+2]
    
    #Z_r
    for i = 1:nrsteps+1
         Z_r[i,j+2] = (2*Z_r[i,j+1]-Z_r[i,j]+(delta_t^2)*(dp_prime_dr[i]-rho_prime[i,j+1]*g_0[i]))/rho_0[i] 
    end
    Z_r[size(Z_r,1),j+2]=Z_r[size(Z_r,1)-1,j+2]-delta_r*(2/r[size(Z_r,1)-1])*Z_r[size(Z_r,1)-1,j+2]
end


In [49]:
nrsteps

200

In [50]:
p_0


202×1 Matrix{Float64}:
 986.879268536886
 NaN
 986.879268536886
 986.6357858642187
 986.2300882044415
 985.662335690271
 984.9327523889816
 984.0416261839621
 982.9893086225552
 981.7762147302776
 980.402822791529
 978.8696740969286
 977.1773726574297
   ⋮
   3.3100546482894075
   2.7115503437587276
   2.17682159606676
   1.7044725950177768
   1.293094475623591
   0.9412663020145853
   0.6475560502228321
   0.4105215892087434
   0.2287116595078222
   0.10066684887910156
   0.024920564342734014
   4.401525826079181e-28

In [52]:
Z_r

202×102 Matrix{Float64}:
 -0.000125663  -0.000125663  NaN           …  NaN            NaN
  0.0           0.0          NaN              NaN            NaN
  0.000125663   0.000125663    4.00016e-6       8.69373e-79    1.07036e-79
  0.000251326   0.000251326    8.87373e-6       2.09333e-78    8.71013e-80
  0.000376986   0.000376986    1.28535e-5       1.10928e-79   -1.25144e-79
  0.000502642   0.000502642    1.67659e-5  …   -9.51913e-79   -1.12917e-79
  0.000628293   0.000628293    2.06876e-5      -8.15737e-79   -1.07774e-80
  0.000753938   0.000753938    2.46282e-5      -6.21171e-80    1.26305e-79
  0.000879575   0.000879575    2.85875e-5       9.92178e-79    2.80901e-79
  0.0010052     0.0010052      3.25639e-5       2.21073e-78    4.48438e-79
  0.00113082    0.00113082     3.65561e-5  …    3.54147e-78    6.29167e-79
  0.00125643    0.00125643     4.05634e-5       4.97082e-78    8.25449e-79
  0.00138203    0.00138203     4.45854e-5       6.50376e-78    1.0408e-78
  ⋮                  